In [1]:
import pandas as pd
import numpy as np 
import xarray as xr 

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib import gridspec, ticker
# from dask.diagnostics import ProgressBar
# import cartopy.crs as ccrs
# import cartopy.feature as cfeature
# import cmocean as cmo
# from cartopy.mpl.geoaxes import GeoAxes
# from mpl_toolkits.axes_grid1 import AxesGrid

/home/bruno/miniconda3/envs/ocean/lib/python3.7/site-packages/xarray/backends/cfgrib_.py:28: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  "Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. "
/home/bruno/miniconda3/envs/ocean/lib/python3.7/site-packages/pyproj/__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


In [2]:
#conda install odfpy

In [3]:
#data = pd.read_excel('/home/bruno/git/cetesb-sbsr/tabela_cetesb')
#pd.read_excel('/home/bruno/git/cetesb-sbsr/Dados_cetesb.xlsb', 
data = pd.read_excel('/home/bruno/git/cetesb-sbsr/cetesb_waterquality_z')

In [4]:
#open data tabela cetesb = tc
tc = pd.DataFrame(data=data)#)

In [5]:
print('Caracteristica das colunas')
tc.info()

Caracteristica das colunas
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68511 entries, 0 to 68510
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Data           68511 non-null  datetime64[ns]
 1   Hora           68511 non-null  object        
 2   Paramt         68511 non-null  object        
 3   Valor          68511 non-null  object        
 4   ValorNumerico  68511 non-null  float64       
 5   Und.           68511 non-null  object        
 6   N. Amostra     67135 non-null  float64       
 7   Sis. Hidrico   68511 non-null  object        
 8   Compartimento  19877 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 4.7+ MB


In [6]:
tc

,Data,Hora,Paramt,Valor,ValorNumerico,Und.,N. Amostra,Sis. Hidrico,Compartimento
0,2019-02-12,11:30:00,Alumínio Dissolvido,0.5,0.50,mg/L,1902762.0,Canal de São Sebastião - Rede Costeira,Superficial
1,2019-02-12,11:30:00,Benzeno,2.5,2.50,µg/L,1902762.0,Canal de São Sebastião - Rede Costeira,Superficial
2,2019-02-12,11:30:00,Boro Total,4.38,4.38,mg/L,1902762.0,Canal de São Sebastião - Rede Costeira,Superficial
3,2019-02-12,11:30:00,Cádmio Total,0.05,0.05,mg/L,1902762.0,Canal de São Sebastião - Rede Costeira,Superficial
4,2019-02-12,11:30:00,Carbono Orgânico Total,1.21,1.21,mg/L,1902762.0,Canal de São Sebastião - Rede Costeira,Superficial
...,...,...,...,...,...,...,...,...,...
68506,2017-09-27,11:00:00,Sólido Suspenso Volátil,100,100.00,mg/L,1723915.0,Rio Itaguaré - Rede Costeira,NaN
68507,2017-09-27,11:00:00,Sólido Total,40504,40504.00,mg/L,1723915.0,Rio Itaguaré - Rede Costeira,NaN
68508,2017-09-27,11:00:00,Sólidos Sedimentáveis,0.1,0.10,mL/L,1723915.0,Rio Itaguaré - Rede Costeira,NaN
68509,2017-09-27,11:00:00,Turbidez,2.6,2.60,UNT,1723915.0,Rio Itaguaré - Rede Costeira,NaN


In [7]:
#verifica qtd de nans
#tc.isnull().sum()

In [8]:
#filtra nans
tc_fn = tc.dropna(axis = 0)

In [9]:
#tc_fn.info()

In [10]:
tc_fn.columns

Index(['Data', 'Hora', 'Paramt', 'Valor', 'ValorNumerico', 'Und.',
       'N. Amostra', 'Sis. Hidrico', 'Compartimento'],
      dtype='object')

In [11]:
cd = tc_fn.loc[:,['Data','Hora','Paramt','ValorNumerico','Und.','Sis. Hidrico','Compartimento']]

In [14]:
# seleção de dados em superficie = ds
ds = cd.loc[cd['Compartimento'] == 'Superficial']

In [64]:
ds

,Data,Hora,Paramt,ValorNumerico,Und.,Sis. Hidrico,Compartimento
0,2019-02-12,11:30:00,Alumínio Dissolvido,0.50,mg/L,Canal de São Sebastião - Rede Costeira,Superficial
1,2019-02-12,11:30:00,Benzeno,2.50,µg/L,Canal de São Sebastião - Rede Costeira,Superficial
2,2019-02-12,11:30:00,Boro Total,4.38,mg/L,Canal de São Sebastião - Rede Costeira,Superficial
3,2019-02-12,11:30:00,Cádmio Total,0.05,mg/L,Canal de São Sebastião - Rede Costeira,Superficial
4,2019-02-12,11:30:00,Carbono Orgânico Total,1.21,mg/L,Canal de São Sebastião - Rede Costeira,Superficial
...,...,...,...,...,...,...,...
42147,2018-12-06,11:05:00,Sólido Suspenso Volátil,100.00,mg/L,Tabatinga - Rede Costeira,Superficial
42148,2018-12-06,11:05:00,Sólido Total,38168.00,mg/L,Tabatinga - Rede Costeira,Superficial
42149,2018-12-06,11:05:00,Sólidos Sedimentáveis,0.50,mL/L,Tabatinga - Rede Costeira,Superficial
42151,2018-12-06,11:05:00,Temperatura da Água,24.10,ºC,Tabatinga - Rede Costeira,Superficial


In [68]:
# seleção de dados canal sao sebastiao = cs_wq
ccs_wq = ds.loc[cd['Sis. Hidrico']== 'Canal de São Sebastião - Rede Costeira']

In [18]:
ccs_wq

,Data,Hora,Paramt,ValorNumerico,Und.,Sis. Hidrico,Compartimento
0,2019-02-12,11:30:00,Alumínio Dissolvido,0.50,mg/L,Canal de São Sebastião - Rede Costeira,Superficial
1,2019-02-12,11:30:00,Benzeno,2.50,µg/L,Canal de São Sebastião - Rede Costeira,Superficial
2,2019-02-12,11:30:00,Boro Total,4.38,mg/L,Canal de São Sebastião - Rede Costeira,Superficial
3,2019-02-12,11:30:00,Cádmio Total,0.05,mg/L,Canal de São Sebastião - Rede Costeira,Superficial
4,2019-02-12,11:30:00,Carbono Orgânico Total,1.21,mg/L,Canal de São Sebastião - Rede Costeira,Superficial
...,...,...,...,...,...,...,...
40360,2018-09-12,12:55:00,Sólido Suspenso Volátil,100.00,mg/L,Canal de São Sebastião - Rede Costeira,Superficial
40361,2018-09-12,12:55:00,Sólido Total,33892.00,mg/L,Canal de São Sebastião - Rede Costeira,Superficial
40362,2018-09-12,12:55:00,Sólidos Sedimentáveis,0.50,mL/L,Canal de São Sebastião - Rede Costeira,Superficial
40364,2018-09-12,12:55:00,Temperatura da Água,21.20,ºC,Canal de São Sebastião - Rede Costeira,Superficial


In [46]:
print(cs_wq['Paramt'])

0            Alumínio Dissolvido
1                        Benzeno
2                     Boro Total
3                   Cádmio Total
4         Carbono Orgânico Total
                  ...           
40360    Sólido Suspenso Volátil
40361               Sólido Total
40362      Sólidos Sedimentáveis
40364        Temperatura da Água
40365                   Turbidez
Name: Paramt, Length: 1958, dtype: object


In [22]:
#aqui acontece seleção por variavel

#seleção de dados de clorofila
chla_a = cs_wq.loc[cd['Paramt'] == 'Clorofila-a' ]

In [23]:
chl_data = chla_a.loc[:, ['Data','Hora','Paramt','ValorNumerico','Und.']]

In [39]:
chl_data

,Data,Hora,Paramt,ValorNumerico,Und.
7,2019-02-12,11:30:00,Clorofila-a,0.56,µg/L
89,2019-02-12,11:39:00,Clorofila-a,0.56,µg/L
182,2019-02-12,12:23:00,Clorofila-a,0.80,µg/L
279,2019-02-12,12:35:00,Clorofila-a,1.19,µg/L
387,2019-02-12,13:08:00,Clorofila-a,0.59,µg/L
509,2019-02-12,13:25:00,Clorofila-a,0.56,µg/L
642,2019-02-12,14:27:00,Clorofila-a,0.56,µg/L
689,2019-02-12,14:29:00,Clorofila-a,0.80,µg/L
746,2019-02-12,15:12:00,Clorofila-a,0.80,µg/L
792,2019-02-12,15:15:00,Clorofila-a,1.13,µg/L


In [60]:
len(chla_a)

50

In [32]:
#seleção de dados de turbidez
turbidez = cs_wq.loc[cd['Paramt'] == 'Turbidez' ]

In [33]:
turbidez = turbidez.loc[:, ['Data','Hora','Paramt','ValorNumerico','Und.']]

In [53]:
turbidez

,Data,Hora,Paramt,ValorNumerico,Und.
45,2019-02-12,11:30:00,Turbidez,1.17,UNT
111,2019-02-12,11:39:00,Turbidez,1.12,UNT
174,2019-02-12,11:47:00,Turbidez,1.17,UNT
220,2019-02-12,12:23:00,Turbidez,1.10,UNT
301,2019-02-12,12:35:00,Turbidez,2.17,UNT
...,...,...,...,...,...
40089,2018-09-12,11:15:00,Turbidez,1.00,UNT
40159,2018-09-12,11:23:00,Turbidez,1.00,UNT
40192,2018-09-12,12:33:00,Turbidez,2.81,UNT
40280,2018-09-12,12:44:00,Turbidez,2.09,UNT


In [61]:
len(turbidez)

75

In [49]:
# Carbono Orgânico Total

#seleção de dados de turbidez
toc = cs_wq.loc[cd['Paramt'] == 'Carbono Orgânico Total' ]

In [59]:
len(toc)

70

In [62]:
toc

,Data,Hora,Paramt,ValorNumerico,Und.,Sis. Hidrico,Compartimento
4,2019-02-12,11:30:00,Carbono Orgânico Total,1.21,mg/L,Canal de São Sebastião - Rede Costeira,Superficial
87,2019-02-12,11:39:00,Carbono Orgânico Total,1.35,mg/L,Canal de São Sebastião - Rede Costeira,Superficial
152,2019-02-12,11:47:00,Carbono Orgânico Total,1.17,mg/L,Canal de São Sebastião - Rede Costeira,Superficial
179,2019-02-12,12:23:00,Carbono Orgânico Total,1.00,mg/L,Canal de São Sebastião - Rede Costeira,Superficial
277,2019-02-12,12:35:00,Carbono Orgânico Total,1.00,mg/L,Canal de São Sebastião - Rede Costeira,Superficial
...,...,...,...,...,...,...,...
40058,2018-09-12,11:15:00,Carbono Orgânico Total,1.00,mg/L,Canal de São Sebastião - Rede Costeira,Superficial
40130,2018-09-12,11:23:00,Carbono Orgânico Total,1.00,mg/L,Canal de São Sebastião - Rede Costeira,Superficial
40160,2018-09-12,12:33:00,Carbono Orgânico Total,1.00,mg/L,Canal de São Sebastião - Rede Costeira,Superficial
40249,2018-09-12,12:44:00,Carbono Orgânico Total,1.00,mg/L,Canal de São Sebastião - Rede Costeira,Superficial


In [54]:
SST = cs_wq.loc[cd['Paramt'] == 'Temperatura da Água']

In [58]:
len(SST)

61

Agora verificando os dados sem classificar por superficie, apenas região (sistema hidrico) para tentar amplificar o numero de amostras

In [78]:
#tc

In [72]:
tc = tc.loc[:,['Data','Hora','Paramt','ValorNumerico','Und.','Sis. Hidrico','Compartimento']]

In [76]:
#canal de são sebastião tabela contendo nan values
ccs = tc.loc[cd['Sis. Hidrico']== 'Canal de São Sebastião - Rede Costeira']

In [77]:
ccs

,Data,Hora,Paramt,ValorNumerico,Und.,Sis. Hidrico,Compartimento
0,2019-02-12,11:30:00,Alumínio Dissolvido,0.50,mg/L,Canal de São Sebastião - Rede Costeira,Superficial
1,2019-02-12,11:30:00,Benzeno,2.50,µg/L,Canal de São Sebastião - Rede Costeira,Superficial
2,2019-02-12,11:30:00,Boro Total,4.38,mg/L,Canal de São Sebastião - Rede Costeira,Superficial
3,2019-02-12,11:30:00,Cádmio Total,0.05,mg/L,Canal de São Sebastião - Rede Costeira,Superficial
4,2019-02-12,11:30:00,Carbono Orgânico Total,1.21,mg/L,Canal de São Sebastião - Rede Costeira,Superficial
...,...,...,...,...,...,...,...
67428,2015-09-22,15:32:00,Sólido Suspenso Volátil,140.00,mg/L,Canal de São Sebastião - Rede Costeira,NaN
67429,2015-09-22,15:32:00,Sólido Total,40648.00,mg/L,Canal de São Sebastião - Rede Costeira,NaN
67430,2015-09-22,15:32:00,Sólidos Sedimentáveis,0.10,mL/L,Canal de São Sebastião - Rede Costeira,NaN
67431,2015-09-22,15:32:00,Temperatura da Água,24.13,ºC,Canal de São Sebastião - Rede Costeira,NaN
